In [ ]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy

from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np


#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jsonlines

In [ ]:
pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.4 MB/s eta 0:00:00


In [ ]:
import pyLDAvis.gensim_models

In [ ]:
df = pd.read_csv("/content/Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()

,ID,Resume_str,Resume_html,Category
1224,26167298,MARKETING CONSULTANT Summar...,"<div class=""fontsize fontface vmargins hmargin...",CONSULTANT
2465,98389424,GUNNERY SERGEANT Core Qua...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
1897,18569929,PAYROLL ACCOUNTANT Summary S...,"<div class=""fontsize fontface vmargins hmargin...",ACCOUNTANT
2254,14626780,GRANTS MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",BANKING
777,18129173,MARKETER / ADMINISTRATOR Prof...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE


In [ ]:
import spacy

spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:

skill_pattern_path = "/content/jz_skill_patterns.jsonl"

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [ ]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [ ]:
data["Clean_Resume"] = clean
data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"] = data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
1224,26167298,MARKETING CONSULTANT Summar...,"<div class=""fontsize fontface vmargins hmargin...",CONSULTANT,marketing consultant summary value creator mar...,"[medium, business, advertising, design, hubspo..."
2465,98389424,GUNNERY SERGEANT Core Qua...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,gunnery sergeant core qualification detail ori...,"[data management, computer network, linux, des..."
1897,18569929,PAYROLL ACCOUNTANT Summary S...,"<div class=""fontsize fontface vmargins hmargin...",ACCOUNTANT,payroll accountant summary sixteen year experi...,"[accounting, stemming, computer programming, m..."
2254,14626780,GRANTS MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",BANKING,grant manager summary finance professional suc...,"[accounting, business, finance, monitoring, co..."
777,18129173,MARKETER / ADMINISTRATOR Prof...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE,marketer administrator professional summary ad...,"[support, marketing, business, ruby]"


In [ ]:
fig = px.histogram(
    data, x="Category", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
Job_Category = data["Category"].unique()
Job_Cat = np.append(Job_Category, "ALL")

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
Total_skills = []
if Job_Cat.all() != "ALL":
    fltr = data[data["Category"] == Job_Cat]["skills"]
    for x in fltr:
        for i in x:
            Total_skills.append(i)
else:
    fltr = data["skills"]
    for x in fltr:
        for i in x:
            Total_skills.append(i)

fig = px.histogram(
    x=Total_skills,
    labels={"x": "Skills"},
    title=f"{Job_Cat} Distribution of Skills",
).update_xaxes(categoryorder="total descending")
fig.show()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
sent = nlp(data["Resume_str"].iloc[0])
displacy.render(sent, style="ent", jupyter=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
displacy.render(sent[0:10], style="dep", jupyter=True, options={"distance": 90})

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
# options=[{"ents": "Job-Category", "colors": "#ff3232"},{"ents": "SKILL", "colors": "#56c426"}]
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp(data["Resume_str"].iloc[7])
displacy.render(sent, style="ent", jupyter=True, options=options)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
docs = data["Clean_Resume"].values
dictionary = corpora.Dictionary(d.split() for d in docs)
bow = [dictionary.doc2bow(d.split()) for d in docs]
lda = gensim.models.ldamodel.LdaModel
num_topics = 4
ldamodel = lda(
    bow,
    num_topics=num_topics,
    id2word=dictionary,
    passes=50,
    minimum_probability=0
)
ldamodel.print_topics(num_topics=num_topics)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



[(0,
  '0.011*"marketing" + 0.009*"state" + 0.009*"company" + 0.009*"city" + 0.008*"management" + 0.006*"name" + 0.006*"project" + 0.005*"business" + 0.005*"development" + 0.005*"new"'),
 (1,
  '0.009*"state" + 0.009*"company" + 0.008*"management" + 0.008*"city" + 0.008*"customer" + 0.008*"service" + 0.007*"name" + 0.006*"sale" + 0.006*"skill" + 0.006*"team"'),
 (2,
  '0.011*"state" + 0.010*"company" + 0.010*"city" + 0.008*"customer" + 0.007*"name" + 0.007*"management" + 0.007*"account" + 0.006*"employee" + 0.006*"financial" + 0.005*"service"'),
 (3,
  '0.013*"food" + 0.011*"customer" + 0.008*"equipment" + 0.007*"state" + 0.007*"city" + 0.006*"service" + 0.006*"work" + 0.006*"company" + 0.005*"product" + 0.005*"name"')]

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(ldamodel, bow, dictionary)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.042032  0.095771       1        1  31.733452
0      0.077835 -0.062672       2        1  28.295232
1      0.047252 -0.021826       3        1  26.820959
3     -0.167119 -0.011273       4        1  13.150357, topic_info=           Term        Freq       Total Category  logprob  loglift
1812       food  236.000000  236.000000  Default  30.0000  30.0000
381   marketing  482.000000  482.000000  Default  29.0000  29.0000
949   equipment  212.000000  212.000000  Default  28.0000  28.0000
186    customer  828.000000  828.000000  Default  27.0000  27.0000
1674    student  204.000000  204.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
413        name   81.014021  845.161229   Topic4  -5.3262  -0.3162
1020     record   53.516541  194.037664   Topic4  -5.7409   0.7407
1002    problem   47.126447  172.848503   Topic4  -5.8680   0.7291
571       skill   49.611667  579.442561   Topic4  -5.8166  -0.4291
605      system   47.397356  498.911145   Topic4  -5.8623  -0.3251

[337 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
71        1  0.848141     account
71        2  0.066459     account
71        3  0.072788     account
71        4  0.012659     account
895       1  0.967933  accountant
...     ...       ...         ...
3527      4  0.982635      wiring
671       1  0.322586        work
671       2  0.190504        work
671       3  0.223524        work
671       4  0.261625        work

[639 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4])